# Combining and analysing the simulations done for testing restarts on SPICE
Using the lowdensity S probe at 10$^{\circ}$

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
# import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

In [3]:
importlib.reload(spl)
importlib.reload(td)
# importlib.reload(hmg)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [2]:
spice_dir = pth.Path('/home/jleland/data/external/spice/')
os.chdir(spice_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/restart_test/sprobe_9hr_multirestart',
    'marconi/restart_test/sprobe_1hr',
    'marconi/restart_test/sprobe_1hr.1',
    'marconi/restart_test/sprobe_1hr_atrestart',
    'marconi/restart_test/sprobe_4hr_multirestart_r',
    'marconi/restart_test/sprobe_4hr_multirestart_2r',
    'marconi/restart_test/sprobe_4hr_multirestart_c',
    'marconi/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/restart_test/sprobe_10hr_multirestart_r',
    'marconi/restart_test/sprobe_10hr_multirestart_2r',
    'marconi/restart_test/sprobe_control_mkomm',
    'marconi/restart_test/sprobe_control_2.13_hdf5',
    'marconi/restart_test/sprobe_20hr_multirestart_2.13_4c',
    'marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c',
    'marconi/restart_test/sprobe_control',
    'marconi/restart_test/sprobe_control_recomp',
    'marconi/restart_test/sprobe_4hr_multirestart_c',
    'marconi/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/restart_test/sprobe_20hr_multirestart_mkomm_4c',
    'marconi/restart_test/sprobe_control_mkomm_rel',
    'marconi/restart_test/sprobe_20hr_multirestart_4c',
    'marconi/restart_test/sprobe_20hr_multirestart_4c.1',
    'marconi/restart_test/sprobe_20hr_multirestart_4c.2',
    'marconi/restart_test/sprobe_20hr_multirestart_4r',
    'marconi/restart_test/sprobe_20hr_multirestart_recomp_4c',
    'marconi/restart_test/sprobe_control_2.13',
    'marconi/restart_test/sprobe_control_2-13-1_hdf5_',
    'marconi/restart_test/sprobe_20hr_multirestart_2-13-1_hdf5_4c',
}
single_sims = {
    'marconi/restart_test/sprobe_control_2.13_',
    'marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c',
    'marconi/restart_test/sprobe_control_2-13-1_',
    'marconi/restart_test/sprobe_20hr_multirestart_2.14_4c',
    'marconi/restart_test/sprobe_1hr_2.14'
}

In [5]:
scans_searchstr = '*/restart_test/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


Single sim marconi/restart_test/sprobe_1hr_2.14 found
marconi/restart_test/sprobe_1hr_2.14: ['marconi/restart_test/sprobe_1hr_2.14']

Single sim marconi/restart_test/sprobe_control_2-13-1_ found
marconi/restart_test/sprobe_control_2-13-1_: ['marconi/restart_test/sprobe_control_2-13-1_']

Single sim marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c found
marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c: ['marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c']

Single sim marconi/restart_test/sprobe_20hr_multirestart_2.14_4c found
marconi/restart_test/sprobe_20hr_multirestart_2.14_4c: ['marconi/restart_test/sprobe_20hr_multirestart_2.14_4c']

Single sim marconi/restart_test/sprobe_control_2.13_ found
marconi/restart_test/sprobe_control_2.13_: ['marconi/restart_test/sprobe_control_2.13_']



In [6]:
skippable = {}
scans = list(scans)
scans.sort()

## Functions for quick analysis of multiple problematic splopters

In [9]:
def plot_raw_data_splopter(path_to_splopter, axes=None, homogenise_fl=False):
    label = str(path_to_splopter).split('/')[-1]
    print(label)
    
    splopter = spl.Splopter(path_to_splopter, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=homogenise_fl, find_se_temp_fl=False)

    # Extract relevant arrays from the matlab file
    probe_indices = splopter.parser.get_probe_obj_indices()
    probe_current_e = 0.0
    probe_current_i = 0.0

    time = np.squeeze(splopter.tdata.t)[:-1]
    for index in probe_indices:
        probe_current_e += np.squeeze(splopter.tdata.objectscurrente)[index]
        probe_current_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
    probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])
    probe_current_tot = probe_current_i + probe_current_e
    
    print(splopter.tdata.diagnostics)
    
    if axes is None:
        fig, axes = plt.subplots(2)
    
    axes[0].set_title(label)
    axes[0].plot(probe_bias, label=label)
#     axes[0].plot(universal_probe_bias)
    axes[0].legend()
    axes[1].plot(probe_current_tot, label=label)
#     axes[1].plot(time, probe_current_i)
#     axes[1].plot(time, probe_current_e)
    axes[1].legend()
    
    return splopter
   

In [10]:
def plot_concatted_current(splopters, trim_concat=False, ax=None):
    label = str(splopters[0].data_dir).split('/')[-2].split('.')[0]
    if label[-1] == 'c':
        trim_concat = True
    concatted_time = np.array([])
    concatted_current = np.array([])
    if trim_concat:
        lengths = []
        
    if ax is None:
        fig, ax = plt.subplots()
    
    for s in splopters:
        # Extract relevant arrays from the matlab file
        probe_indices = s.parser.get_probe_obj_indices()
        probe_current_e = 0.0
        probe_current_i = 0.0

        time = np.squeeze(s.tdata.t)[:-1]
        if len(concatted_time) > 0:
            offset = concatted_time[-1]
        else:
            offset = 0
        
        for index in probe_indices:
            probe_current_e += np.squeeze(s.tdata.objectscurrente)[index]
            probe_current_i += np.squeeze(s.tdata.objectscurrenti)[index]
        probe_bias = np.squeeze(s.tdata.diagnostics['ProbePot'])
        probe_current_tot = probe_current_i + probe_current_e
        
        if trim_concat:
            indices_oi = np.where((probe_current_tot != 0.0) & (probe_current_tot <= 100.0) & (probe_current_tot >= -500.0))
            
            concatted_current = np.concatenate([concatted_current, probe_current_tot[indices_oi]])
            concatted_time = np.concatenate([concatted_time, offset + np.arange(len(time[indices_oi]))])
            
            ax.axvline(x=offset, **c.AX_LINE_DEFAULTS)
            lengths.append(offset)
        else:
            print(f'time: {len(time)}, current: {len(probe_current_tot)}')
            concatted_current = np.concatenate([concatted_current, probe_current_tot])
            concatted_time = np.concatenate([concatted_time, offset + time])
    
    if trim_concat:
        ax.plot(concatted_time, concatted_current, label=label)
        return concatted_time, concatted_current
    else:
        ax.plot(concatted_time, concatted_current, label=label)
        return concatted_time, concatted_current

In [11]:
def plot_concatted_bias(splopters, trim_concat=False, ax=None):
    label = str(splopters[0].data_dir).split('/')[-2].split('.')[0]
    
    concatted_time = np.array([])
    concatted_voltage = np.array([])
    if trim_concat:
        lengths = []
        
    if ax is None:
        fig, ax = plt.subplots()
    
    for s in splopters:
        probe_bias = np.squeeze(s.tdata.diagnostics['ProbePot'])
#         print(probe_bias.shape)
        if trim_concat:
            concatvolt_len = len(concatted_voltage)
            concatted_voltage = np.concatenate([concatted_voltage, probe_bias[concatvolt_len:]])
            ax.axvline(x=concatvolt_len, **c.AX_LINE_DEFAULTS)
            lengths.append(concatvolt_len)
        elif isinstance(probe_bias, np.ndarray) and len(probe_bias.shape) > 0:
            concatted_voltage = np.concatenate([concatted_voltage, probe_bias])
    
    ax.plot(concatted_voltage, label=label)
    return concatted_voltage

## Cells for selecting and comparing different combinations of broken splopters
The options are wide ranging and, as of 2019-10-21, consist of:
 - Hard time limits of 1, 2, 4 and no hours
 - Different simulation restart methods: -c or -r (i.e. full or partial restart)
 - Manual restart using -c (i.e. editing the slurm file manually and using qsub/sbatch)
 - Michael Komm's off-repository src/ folder for SPICE

In [8]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/restart_test/sprobe_1hr_2.14
1 marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c
2 marconi/restart_test/sprobe_20hr_multirestart_2.14_4c
3 marconi/restart_test/sprobe_control_2-13-1_
4 marconi/restart_test/sprobe_control_2.13_


In [9]:
print(desired_variables)

{'objectsenum', 'count', 'm', 'Nzmax', 'Nz', 'dz', 'dt', 'alphayz', 'q', 'objectspowerfluxi', 'Ny', 't', 'snumber', 'objectscurrente', 'Potvac', 'alphaxz', 'nproc', 'Npc', 'Pot', 'npartproc', 'ProbePot', 'Temp', 'objectspowerfluxe', 'objectscurrenti'}


In [10]:
chosen_spl_path = spice_dir / all_run_dirs[scans[1]][0]
print(chosen_spl_path)

splopter = spl.Splopter(chosen_spl_path, reduce=desired_variables)
splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

/home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109/t-sprobe_20hr_multirestart_2-13-1_4c.mat
No a-file given, continuing without
Loading backup backup_20191105-2109 (1 of 5) for current and bias concatenation
Loading backup backup_20191106-0119 (2 of 5) for current and bias concatenation
Stripping head for backup 2 of 5
Loading backup backup_20191106-0159 (3 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 3 of 5
Stripping head for backup 3 of 5
Found an odd number of spike transitions


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
plot_raw_data_splopter(chosen_spl_path)

sprobe_20hr_multirestart_2-13-1_4c
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109/t-sprobe_20hr_multirestart_2-13-1_4c.mat
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}


In [17]:
backups = list(chosen_spl_path.glob('backup*'))
backups.sort()

bu_splopters = []
for i, bu_folder in enumerate(backups):
    print(f'{i}) {str(bu_folder)}')
    
    splopter = spl.Splopter(bu_folder, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
    bu_splopters.append(splopter)
    

0) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109
No a-file given, continuing without
1) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0119
No a-file given, continuing without
2) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0159
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
3) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0213
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
4) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0228
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without


In [18]:
print(plot_concatted_current(bu_splopters, trim_concat=True))

(array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.602e+03, 1.603e+03,
       1.604e+03]), array([-1.25034001e-01, -1.25034001e-01, -1.25034001e-01, ...,
       -3.08168203e+02, -3.03596808e+02, -2.95385452e+02]))


In [19]:
fig, axes = plt.subplots(2)
for bu_folder in backups:
#     plot_raw_data_splopter(bu_folder, axes=axes)
    plot_raw_data_splopter(bu_folder)
# axes[1].set_ylim(-100, 100)

backup_20191105-2109
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
backup_20191106-0119
No a-file given, continuing without
{'ProbePot': array([[-9.9 ],
       [-9.9 ],
       [-9.9 ],
       ...,
       [ 7.75],
       [ 7.75],
       [ 7.75]])}
backup_20191106-0159
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
{'ProbePot': array([[-9.9 ],
       [-9.9 ],
       [-9.9 ],
       ...,
       [10.05],
       [10.05],
       [10.05]])}
backup_20191106-0213
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [ 0. ],
       [ 0. ],
       [-9.9]])}
backup_20191106-0228
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
{'Probe

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


## Comparing different hard time limited simulations

In [28]:
print([scan for scan in scans if '10hr' in scan])

[]


In [45]:
fig, axes = plt.subplots(2)
for scan in scans:
    label = scan.split('/')[-1]
    if '10hr' not in label or 'c' in label:
        print(f'Skipping "{label}"...')
        continue
    plot_raw_data_splopter(scan, axes=axes)

sprobe_10hr_multirestart_r
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
Skipping "sprobe_4hr_multirestart_r"...
Skipping "sprobe_4hr_multirestart_2c"...
Skipping "sprobe_4hr_multirestart_c"...
Skipping "sprobe_4hr_multirestart_2r"...
sprobe_10hr_multirestart_2r
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
Skipping "sprobe_control"...


In [57]:
bu_splopters = {}

for bu_folder in backups:
    splopter = spl.Splopter(bu_folder, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
    
    bu_splopters[bu_folder] = splopter

No a-file given, continuing without
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without


In [68]:
for bu_s in bu_splopters.values():
    print(bu_s.data_dir)

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0032
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0133
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0233
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0334
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0434
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0535
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0635
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0736
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0838


In [72]:
print(plot_concatted_current(bu_splopters.values()))

[0, 1296, 1296, 1769, 1769, 1769, 1769, 1800, 1800]


## Looking at -r restart sims

In [78]:
scans = list(scans)
scans.sort()
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/restart_test/sprobe_20hr_multirestart_4c
1 marconi/restart_test/sprobe_20hr_multirestart_4c.1
2 marconi/restart_test/sprobe_20hr_multirestart_4c.2
3 marconi/restart_test/sprobe_20hr_multirestart_4r
4 marconi/restart_test/sprobe_20hr_multirestart_mkomm_4c
5 marconi/restart_test/sprobe_20hr_multirestart_recomp_4c
6 marconi/restart_test/sprobe_control
7 marconi/restart_test/sprobe_control_mkomm_rel
8 marconi/restart_test/sprobe_control_recomp


In [20]:
# scans_comparison = [scans[6], scans[4], scans[3],]
scans_comparison = [scans[1], scans[0], scans[4],]
# scans_comparison = [scan for scan in scans if 'control' in scan]
# scans_comparison = [scan for scan in scans if '20hr' in scan]
# scans_comparison = [scan for scan in scans if scan[-1] == 'c' or ('control' in scan and 'mkomm' not in scan)]
scans_comparison.sort(reverse=True)
print(scans_comparison)

['marconi/restart_test/sprobe_control', 'marconi/restart_test/sprobe_20hr_multirestart_4c.1', 'marconi/restart_test/sprobe_20hr_multirestart_4c']


In [21]:
fig, axes = plt.subplots(2)
# axes_twinned = [axes, [ax.twinx() for ax in axes]]

for i, scan in enumerate(scans_comparison):
    plot_raw_data_splopter(scan, axes=axes)

sprobe_control
{'ProbePot': array([[-9.9 ],
       [-9.9 ],
       [-9.9 ],
       ...,
       [10.05],
       [10.05],
       [10.05]])}
sprobe_20hr_multirestart_4c.1
Looking for a suitable backup to use instead.
Useable backup found at marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0357/t-sprobe_20hr_multirestart_4c.1.mat
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
sprobe_20hr_multirestart_4c
Looking for a suitable backup to use instead.
Useable backup found at marconi/restart_test/sprobe_20hr_multirestart_4c/backup_20191022-0054/t-sprobe_20hr_multirestart_4c.mat
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}


In [22]:
fig, axes_whole = plt.subplots(3, 2)

for i, scan in enumerate(scans_comparison):
    axes = axes_whole[i]
    
    backups = list(pth.Path(scan).glob('backup*'))
    backups.sort()
    
    bu_splopters = []
    for j, bu_folder in enumerate(backups):
        print(f'{j}) {str(bu_folder)}')

        splopter = spl.Splopter(bu_folder, reduce=desired_variables)
        splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
        bu_splopters.append(splopter)
    
    plot_concatted_current(bu_splopters, ax=axes[0])
    plot_concatted_bias(bu_splopters, ax=axes[1])
    axes[0].set_title(scan.split('/')[-1])
    axes[0].legend()
    axes[1].legend()
    

0) marconi/restart_test/sprobe_control/backup_20191018-0758
time: 1110, current: 1110
0) marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0357
No a-file given, continuing without
1) marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0808
2) marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0856
There are no backups to utilise. Carrying on, but expect erroneous results.
3) marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0910
There are no backups to utilise. Carrying on, but expect erroneous results.
4) marconi/restart_test/sprobe_20hr_multirestart_4c.1/backup_20191023-0925
There are no backups to utilise. Carrying on, but expect erroneous results.
time: 1169, current: 1169
time: 1053, current: 1053
time: 1110, current: 1110
time: 1110, current: 1110
time: 1110, current: 1110
0) marconi/restart_test/sprobe_20hr_multirestart_4c/backup_20191022-0054
No a-file given, continuing without
1) marconi/restart_test/sprobe_20hr_

## Look into difference between different compilers
Two different versions of intelmpi were used for exactly the same input. Input was:
 - S-probe
 - 10$^\circ$ incidence angle
 - 4 hour hard time limit
 - Run on Marconi
 - Hard restart mode (-c)
 - spice2-legacy repository (unaltered) 

The only difference was the version of intelmpi used for compilation and exexution (mpirun) used in each case:
 - Compiled with 2018, run with 2018 - sprobe_20hr_multirestart_recomp_4c
 - Compiled with 2017, run with 2018 - sprobe_20hr_multirestart_4c.2
 - Compiled with 2018, run with 2017 - N/A
 - Compiled with 2017, run with 2017 - sprobe_20hr_multirestart_4c & sprobe_20hr_multirestart_4c.1 
 
 
An intermediate combination of "Compiled with 2017, run with 2018" will be attempted. The converse (comp /w 2018, run /w 2017) is not possible. 

In [17]:
scan = list(scans)
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/restart_test/sprobe_control_2.13_
1 marconi/restart_test/sprobe_control_2-13-1_
2 marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c


In [20]:
# scans_comparison = [scans[0], scans[2], scans[5]]
scans_comparison = [scans[0], scans[1], scans[2]]

In [21]:
fig, axes = plt.subplots(2)
fig_whole, axes_whole = plt.subplots(3, 2)

splopters = []
times = []
currents = []
voltages = []

for i, scan in enumerate(scans_comparison):
    print(scan)
    s = plot_raw_data_splopter(scan, axes=axes)
    splopters.append(s)
    
    ax = axes_whole[i]
    
    backups = list(pth.Path(scan).glob('backup*'))
    backups.sort()
    
    bu_splopters = []
    for j, bu_folder in enumerate(backups):
        print(f'{j}) {str(bu_folder)}')

        splopter = spl.Splopter(bu_folder, reduce=desired_variables)
        splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
        bu_splopters.append(splopter)
    
    t, i = plot_concatted_current(bu_splopters, ax=ax[0])
    v = plot_concatted_bias(bu_splopters, ax=ax[1])
    
    ax[0].set_title(scan.split('/')[-1])
    ax[0].legend()
    ax[1].legend()
    
    times.append(t)
    currents.append(i)
    voltages.append(v)
    

marconi/restart_test/sprobe_control_2.13_
sprobe_control_2.13_
Spice data directory is not valid, attempting to auto-fix.
/home/jleland/Spice/spice2/marconi/restart_test/sprobe_control_2.13_ is not a directory


ValueError: Passed Spice directory (marconi/restart_test/sprobe_control_2.13_) is not valid.

In [63]:
dts = []
for i in range(len(splopters)):
    print(f'\n ===== splopter {i} =====')
    print(f'len(t) = {len(times[i])}')
    print(f'len(I) = {len(currents[i])}')
    print(f'dt = {np.squeeze(splopters[i].tdata.dt):.4g}')
    dts.append(np.squeeze(splopters[i].tdata.dt))


 ===== splopter 0 =====
len(t) = 1600
len(I) = 1600
dt = 0.0003396

 ===== splopter 1 =====
len(t) = 1606
len(I) = 1606
dt = 0.0003396

 ===== splopter 2 =====
len(t) = 1110
len(I) = 1110
dt = 0.0003396


In [85]:
0.17388301 / 0.0003396

512.0229976442874

In [64]:
plt.figure()
plt.plot(times[0] * dts[0], currents[0])
plt.plot(times[1] * dts[1], currents[1])
plt.plot(times[2], currents[2])

In [76]:
fig, axes = plt.subplots(3, sharex=True)

for i in range(3):
    axes[0].plot(times[i])
axes[0].plot(np.squeeze(splopters[0].tdata.t)[:-1])

for i in range(3):
    axes[1].plot(np.diff(times[i]))
print(times[2])
    

axes[2].plot(times[0])
axes[2].plot(times[0] * 0.17388301)
axes[2].plot(times[1])
axes[2].plot(times[1] * 0.17388301)
axes[2].plot(times[2])


[1.73883010e-01 3.47766021e-01 5.21649031e-01 ... 1.92662376e+02
 1.92836259e+02 1.93010142e+02]


In [58]:
fig, ax = plt.subplots()

new_time = np.squeeze(splopters[0].tdata.t)[:-1]
print(new_time.shape)
ax.plot(new_time)

(1167,)


In [68]:
for i in range(len(splopters)):
    for bu in splopters[i].backup_folders:
        print(bu)

marconi/restart_test/sprobe_20hr_multirestart_4c.2/backup_20191023-2054
marconi/restart_test/sprobe_20hr_multirestart_4c.2/backup_20191024-0104
marconi/restart_test/sprobe_20hr_multirestart_4c.2/backup_20191024-0154
marconi/restart_test/sprobe_20hr_multirestart_4c.2/backup_20191024-0208
marconi/restart_test/sprobe_20hr_multirestart_4c.2/backup_20191024-0223
marconi/restart_test/sprobe_20hr_multirestart_recomp_4c/backup_20191022-1315
marconi/restart_test/sprobe_20hr_multirestart_recomp_4c/backup_20191022-1727
marconi/restart_test/sprobe_20hr_multirestart_recomp_4c/backup_20191022-1814
marconi/restart_test/sprobe_20hr_multirestart_recomp_4c/backup_20191022-1829
marconi/restart_test/sprobe_20hr_multirestart_recomp_4c/backup_20191022-1845
marconi/restart_test/sprobe_control_recomp/backup_20191022-1832


# Comparison of versions 2.13 and 2.14

This was run after creating an unhomogenised splopter for the 2.14 run

In [11]:
chosen_spl_path

PosixPath('/home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c')

In [12]:
label = str(chosen_spl_path).split('/')[-1]
# print(label)

# splopter = spl.Splopter(path_to_splopter, reduce=desired_variables)
# splopter.prepare(denormaliser_fl=False, homogenise_fl=homogenise_fl, find_se_temp_fl=False)

# Extract relevant arrays from the matlab file
probe_indices = splopter.parser.get_probe_obj_indices()
probe_current_e = 0.0
probe_current_i = 0.0

time = np.squeeze(splopter.tdata.t)[:-1]
for index in probe_indices:
    probe_current_e += np.squeeze(splopter.tdata.objectscurrente)[index]
    probe_current_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])
probe_current_tot = probe_current_i + probe_current_e

print(splopter.tdata.diagnostics)

fig, axes = plt.subplots(2)

axes[0].set_title(label)
axes[0].plot(probe_bias, label=label)
#     axes[0].plot(universal_probe_bias)
axes[0].legend()
axes[1].plot(probe_current_tot, label=label)
#     axes[1].plot(time, probe_current_i)
#     axes[1].plot(time, probe_current_e)
axes[1].legend()

{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}


In [13]:
backups = list(chosen_spl_path.glob('backup*'))
backups.sort()

bu_splopters = []
for i, bu_folder in enumerate(backups):
    print(f'{i}) {str(bu_folder)}')
    
    splopter = spl.Splopter(bu_folder, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
    bu_splopters.append(splopter)

0) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109
No a-file given, continuing without
1) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0119
No a-file given, continuing without
2) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0159
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
3) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0213
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
4) /home/jleland/data/external/spice/marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0228
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without


In [14]:
# fig, axes = plt.subplots(2)
for bu_folder in backups:
#     plot_raw_data_splopter(bu_folder, axes=axes)
    plot_raw_data_splopter(bu_folder)
# axes[1].set_ylim(-100, 100)

NameError: name 'plot_raw_data_splopter' is not defined